In [4]:
import praw
from prawcore.exceptions import NotFound, Forbidden
import datetime
import csv
import pandas as pd
import os

# Fill below fields
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    redirect_uri="",
    password="",
    user_agent="",
    username="",
)
print(reddit.user.me())

Ill_Action_9778


# Explore User Features

In [8]:
import pprint
usr = reddit.redditor("Ill_Action_9778")

In [11]:
print(usr.name)

Ill_Action_9778


In [12]:
pprint.pprint(vars(usr))

{'_fetched': False,
 '_listing_use_sort': True,
 '_reddit': <praw.reddit.Reddit object at 0x0000015F2B62D290>,
 'name': 'Ill_Action_9778'}


Interesting fields: (link_karma, comment_karma, is_blocked, is_suspended, is_mod)

# Update with new usrs

In [25]:
old_usr_df = pd.read_csv("users.csv")
new_usr_df = pd.read_csv("users.csv")['User']
print(f"# old users: {old_usr_df.shape}")

usr_df_list = [old_usr_df['User']]
for _,_,files in os.walk("."):
    for file in files:
        if file.endswith("_new.csv"):
            print(f"Working on {file}")
            usr_df_list.append(pd.read_csv(file)['author'])

new_usr_df = pd.concat(usr_df_list).drop_duplicates(keep=False)
print(f"# new users: {new_usr_df.shape}")
# new_usr_df.to_csv("users.csv", index=False)

# old users: (1260669, 1)
Working on political_subreddits_new.csv
Working on popular_subreddits_new.csv
Working on sus_usr_subreddits_0_new.csv
Working on sus_usr_subreddits_10_new.csv
Working on sus_usr_subreddits_1_new.csv


C:\Users\Melissa\AppData\Local\Temp\ipykernel_19500\7826499.py:10: DtypeWarning: Columns (3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  usr_df_list.append(pd.read_csv(file)['author'])


Working on sus_usr_subreddits_2_new.csv
Working on sus_usr_subreddits_3_new.csv
Working on sus_usr_subreddits_4_new.csv
Working on sus_usr_subreddits_5_new.csv
Working on sus_usr_subreddits_6_new.csv
Working on sus_usr_subreddits_7_new.csv
Working on sus_usr_subreddits_8_new.csv
Working on sus_usr_subreddits_9_new.csv
# new users: (0,)


In [40]:
old_usr_df.iloc[0:0]

,User


# Get users from mega 100000 list

This should be changed to whatever source has a list of users you wish to fetch info of.

In [16]:
path = os.path.join("output", "mega", "mega_bip_100000_nodes.csv")
nodes_df = pd.read_csv(path)
usrs = nodes_df[nodes_df['d16'] == 'user']['Id']
usrs

1                    shomyo
3                  Whey-Men
5                keypuncher
7        FrontpageWatch2020
9        intelligentreviews
                ...        
66925          WillKuzunoha
66926    VegetableSeason100
66927    GoldEmbarrassed191
66928     decolonialcypriot
66929           Watchdogs66
Name: Id, Length: 66116, dtype: object

# Get usr features

In [20]:

def save_data_list(data_list, csv_name):
    print(f"[{datetime.datetime.now()}] Saving {len(data_list)} entries...")
    df = pd.DataFrame(data_list)
    df.drop_duplicates(subset="User", keep="last", inplace=True)
    # Display the DataFrame
    if os.path.isfile(f"{csv_name}.csv"):
        df.to_csv(f"{csv_name}.csv", mode='a', header=False, index=False)
    else:
        df.to_csv(f"{csv_name}.csv", index=False)
    # Erase to save memory
    data_list.clear()
    df.iloc[0:0]

def extract_usr_features(usr_name):
    usr_feat = None

    usr = reddit.redditor(usr_name)
    # Redditor attributes: https://praw.readthedocs.io/en/latest/code_overview/models/redditor.html#redditor
    try:
        usr_feat = {
            'User': usr_name,
            'link_karma': usr.link_karma,
            'comment_karma': usr.comment_karma,
            'is_blocked': 1 if usr.is_blocked else 0,
            'is_mod': 1 if usr.is_mod else 0
        }
        try:
            usr_feat['is_suspended'] = 1 if usr.is_suspended else 0 # might not have?
        except:
            usr_feat['is_suspended'] = None
    except Exception as e:
        print(f"[{datetime.datetime.now()}] Skipping adding this comment due to: {e}")
             
    return usr_feat

In [21]:
# usr_df = pd.read_csv("users.csv")
# # new_usr_names = new_usr_df.tolist()
# new_usr_names = usr_df['User'].tolist()
new_usr_feat = []
usr_cnt = 0
if len(usrs) > 0:
    for usr_name in usrs:
        usr_cnt += 1
        print(f"[{datetime.datetime.now()}] [{usr_cnt}/{len(usrs)}] Working on {usr_name}")
        usr_feat = extract_usr_features(usr_name)
        if usr_feat is not None:
            new_usr_feat.append(usr_feat)
        if usr_cnt % 100 == 0:
            save_data_list(new_usr_feat, "users_feats.csv")
            new_usr_feat = []
    save_data_list(new_usr_feat, "users_feats.csv")
    new_usr_feat = []
else:
    print("No new users found! Done.")

[2023-11-25 16:43:32.423503] [1/66116] Working on shomyo
[2023-11-25 16:43:32.931785] [2/66116] Working on Whey-Men
[2023-11-25 16:43:33.182906] [3/66116] Working on keypuncher
[2023-11-25 16:43:33.375901] [4/66116] Working on FrontpageWatch2020
[2023-11-25 16:43:33.578767] [5/66116] Working on intelligentreviews
[2023-11-25 16:43:33.808299] [6/66116] Working on Barch3
[2023-11-25 16:43:34.010625] [7/66116] Working on anarchyart2021
[2023-11-25 16:43:34.214785] [8/66116] Working on Down-not-out
[2023-11-25 16:43:34.431898] [9/66116] Working on ETHCrosspostBot
[2023-11-25 16:43:34.644798] [10/66116] Working on One-Yam2819
[2023-11-25 16:43:34.886547] [11/66116] Working on azcoinnews
[2023-11-25 16:43:35.129188] [12/66116] Working on Poke-Oji
[2023-11-25 16:43:35.335813] [13/66116] Working on News-Flunky
[2023-11-25 16:43:35.551924] [14/66116] Working on evaldez14
[2023-11-25 16:43:35.781993] [15/66116] Working on Signal_Lemon_8926
[2023-11-25 16:43:36.065987] [16/66116] Working on GDT_B

KeyboardInterrupt: 